In [7]:
from PIL import Image
import tensorflow as tf
import numpy as np
import os

imagePath = "/HDD2/HnE_images/img_Classification/TCGA/LGG/sep_512/codel/"
imagePathList = []
for subfolder1 in os.listdir(imagePath):
    for subfolder2 in os.listdir(imagePath+subfolder1):
        for imagefile in os.listdir(imagePath+subfolder1+'/'+subfolder2):
            imagePathList.append(imagePath+subfolder1+'/'+subfolder2+'/'+imagefile)

imagePathArray = np.random.permutation(imagePathList)
imageSizeList = [2, 3, 4, 5, 6, 7, 8]

tfrOpt = tf.io.TFRecordOptions(compression_type=None)
tfrWriterDict = {}

for imageSize in imageSizeList:
    tfrWriterDict[imageSize] = tf.io.TFRecordWriter("ODG-tfrec-100-r0%d.tfrecords"%imageSize)

for imagePath in imagePathArray[:100]:
    img = Image.open(imagePath)
    for imageSize in imageSizeList:
        quant = np.array(img.resize((2**imageSize, 2**imageSize), Image.ANTIALIAS))
        ex = tf.train.Example(features=tf.train.Features(feature={
                        'shape': tf.train.Feature(int64_list=tf.train.Int64List(value=quant.shape)),
                        'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tobytes()]))}))
        tfrWriterDict[imageSize].write(ex.SerializeToString())

In [105]:
pixel_size = 8
tfr_opt = tf.io.TFRecordOptions(compression_type=None)
tfr_writer = tf.io.TFRecordWriter("./a.tfrecords", tfr_opt)
for i in range(11400):
    quant = np.array(np.random.normal(np.ones((pixel_size,pixel_size,3)))*255, dtype=np.uint8)
    ex = tf.train.Example(features=tf.train.Features(feature={
                    'shape': tf.train.Feature(int64_list=tf.train.Int64List(value=quant.shape)),
                    'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tobytes()]))}))
    tfr_writer.write(ex.SerializeToString())

In [ ]:
img = np.asarray(PIL.Image.open(io.BytesIO(value)))

In [5]:
def parse_tfrecord_tf(record):
    features = tf.io.parse_single_example(record, features={
        'shape': tf.io.FixedLenFeature([3], tf.int64),
        'data': tf.io.FixedLenFeature([], tf.string)})
    raw_data = tf.io.decode_raw(features['data'], tf.uint8)
    float_data = tf.cast(tf.reshape(raw_data, features['shape']), dtype=tf.float32)
    data = float_data / 128.5 -1
    return data

In [8]:
buffer_mb       = 256
train_dataset = tf.data.TFRecordDataset('Molpax/ODG-tfrec-100-r08.tfrecords', compression_type='', buffer_size=buffer_mb<<20)
BATCH_SIZE = [128, 64, 64, 32, 32, 16, 8]
train_dataset = train_dataset.map(parse_tfrecord_tf)
for train_data in train_dataset:
    print(train_data)

tf.Tensor(
[[[ 0.6342412  -0.33073932 -0.17509729]
  [ 0.6342412  -0.2529183  -0.11284047]
  [ 0.07392991 -0.5330739  -0.05058366]
  ...
  [ 0.74319065 -0.04280156  0.24513614]
  [ 0.44747078 -0.37743193  0.07392991]
  [ 0.6031128  -0.41634244 -0.2140078 ]]

 [[ 0.40077817 -0.49416345 -0.12062258]
  [ 0.4941634  -0.42412454 -0.2373541 ]
  [ 0.5330739  -0.32295722 -0.10505837]
  ...
  [ 0.4708171  -0.48638135 -0.2217899 ]
  [ 0.4708171  -0.47081715 -0.18287939]
  [ 0.4941634  -0.38521403 -0.16731519]]

 [[ 0.27626455 -0.40856034  0.00389099]
  [ 0.27626455 -0.38521403 -0.06614786]
  [ 0.29182875 -0.5175097  -0.19844359]
  ...
  [ 0.32295716 -0.48638135 -0.14396888]
  [ 0.08949411 -0.6031128  -0.18287939]
  [ 0.6031128  -0.36186773 -0.04280156]]

 ...

 [[ 0.18287933 -0.45525295 -0.06614786]
  [ 0.29182875 -0.40856034 -0.12840468]
  [ 0.39299607 -0.39299613 -0.06614786]
  ...
  [-0.2451362  -0.78210115 -0.43190664]
  [-0.41634244 -0.81322956 -0.47859925]
  [-0.38521403 -0.8910506  -0.494

tf.Tensor(
[[[ 0.0583657  -0.40856034 -0.08171207]
  [ 0.10505831 -0.42412454 -0.06614786]
  [ 0.33852136 -0.2062257   0.12062252]
  ...
  [ 0.76653695  0.41634238  0.540856  ]
  [ 0.68093383  0.30739295  0.5175097 ]
  [ 0.27626455 -0.19066149  0.24513614]]

 [[ 0.11284041 -0.33852142  0.09727621]
  [ 0.17509723 -0.36964983  0.12062252]
  [ 0.31517506 -0.15175098  0.27626455]
  ...
  [ 0.44747078 -0.02723736  0.28404665]
  [ 0.37743187 -0.11284047  0.21400774]
  [ 0.08949411 -0.38521403  0.07392991]]

 [[ 0.26070035 -0.2373541   0.28404665]
  [ 0.29961085 -0.15175098  0.23735404]
  [ 0.35408556 -0.00389105  0.29182875]
  ...
  [ 0.26070035 -0.08949417  0.26848245]
  [ 0.24513614 -0.11284047  0.22178984]
  [ 0.18287933 -0.14396888  0.25291824]]

 ...

 [[ 0.4785992  -0.01945525  0.38521397]
  [ 0.34630346 -0.08171207  0.30739295]
  [ 0.12062252 -0.2762646   0.12840462]
  ...
  [ 0.14396882 -0.2373541   0.21400774]
  [ 0.07392991 -0.36964983  0.09727621]
  [-0.04280156 -0.37743193  0.011

tf.Tensor(
[[[ 0.68093383  0.4785992   0.71206224]
  [ 0.41634238  0.30739295  0.5797665 ]
  [ 0.5797665   0.22178984  0.5097276 ]
  ...
  [ 0.6264591   0.4785992   0.69649804]
  [ 0.6264591   0.24513614  0.463035  ]
  [ 0.42412448  0.31517506  0.5175097 ]]

 [[ 0.68093383  0.43190658  0.5564202 ]
  [ 0.36964977  0.22178984  0.5097276 ]
  [ 0.33073926 -0.05058366  0.26848245]
  ...
  [ 0.5175097   0.41634238  0.6575875 ]
  [ 0.70428014  0.35408556  0.5719844 ]
  [ 0.32295716  0.09727621  0.20622563]]

 [[ 0.43190658  0.10505831  0.33852136]
  [ 0.15953302 -0.01945525  0.33073926]
  [ 0.19066143 -0.16731519  0.19844353]
  ...
  [ 0.6420233   0.4785992   0.75875485]
  [ 0.79766536  0.5564202   0.82879376]
  [ 0.27626455  0.0505836   0.33073926]]

 ...

 [[ 0.35408556 -0.02723736  0.30739295]
  [ 0.25291824 -0.02723736  0.36186767]
  [ 0.26070035 -0.01945525  0.32295716]
  ...
  [ 0.81322956  0.69649804  0.68093383]
  [ 0.4708171   0.29961085  0.5252918 ]
  [ 0.6420233   0.5564202   0.774

tf.Tensor(
[[[ 0.80544746  0.18287933  0.69649804]
  [ 0.84435797  0.22957194  0.75875485]
  [ 0.30739295 -0.31517512  0.22957194]
  ...
  [ 0.72762644  0.26848245  0.5642023 ]
  [ 0.8910506   0.40856028  0.72762644]
  [ 0.5564202   0.07392991  0.41634238]]

 [[ 0.68093383  0.0272373   0.540856  ]
  [ 0.68093383  0.0428015   0.5719844 ]
  [ 0.33852136 -0.2918288   0.24513614]
  ...
  [ 0.6031128   0.12840462  0.4708171 ]
  [ 0.6031128   0.09727621  0.4785992 ]
  [ 0.4785992  -0.02723736  0.37743187]]

 [[ 0.43190658 -0.2373541   0.26070035]
  [ 0.44747078 -0.2140078   0.29961085]
  [ 0.43968868 -0.2062257   0.32295716]
  ...
  [ 0.5564202   0.07392991  0.463035  ]
  [ 0.5175097  -0.00389105  0.41634238]
  [ 0.5097276  -0.00389105  0.44747078]]

 ...

 [[ 0.5175097  -0.10505837  0.36964977]
  [ 0.5719844  -0.07392997  0.43190658]
  [ 0.33852136 -0.34630352  0.19844353]
  ...
  [ 0.5486381   0.01945519  0.463035  ]
  [ 0.4863813  -0.00389105  0.41634238]
  [ 0.6264591   0.15175092  0.540

tf.Tensor(
[[[ 0.25291824 -0.47859925 -0.15953308]
  [ 0.19844353 -0.5019455  -0.19066149]
  [ 0.29961085 -0.40856034 -0.01167315]
  ...
  [ 0.21400774 -0.47081715 -0.12062258]
  [-0.00389105 -0.5953307  -0.08171207]
  [ 0.13618672 -0.39299613  0.0272373 ]]

 [[ 0.43190658 -0.307393   -0.01945525]
  [ 0.44747078 -0.36186773 -0.03501946]
  [ 0.40077817 -0.34630352 -0.07392997]
  ...
  [ 0.22957194 -0.43968874 -0.13618678]
  [-0.03501946 -0.6031128  -0.19066149]
  [ 0.08949411 -0.45525295 -0.08171207]]

 [[ 0.23735404 -0.43968874 -0.08171207]
  [ 0.35408556 -0.36186773  0.0428015 ]
  [ 0.33852136 -0.36186773  0.00389099]
  ...
  [ 0.33852136 -0.40077823 -0.00389105]
  [ 0.07392991 -0.5797665  -0.11284047]
  [ 0.08949411 -0.540856   -0.16731519]]

 ...

 [[ 0.35408556 -0.15175098  0.18287933]
  [ 0.31517506 -0.17509729  0.18287933]
  [ 0.5330739  -0.12840468  0.26070035]
  ...
  [ 0.5642023   0.0272373   0.4785992 ]
  [ 0.5564202   0.07392991  0.4785992 ]
  [ 0.6342412   0.17509723  0.439

tf.Tensor(
[[[ 0.12062252 -0.307393    0.10505831]
  [ 0.5642023   0.08171201  0.44747078]
  [ 0.71984434  0.22178984  0.5097276 ]
  ...
  [ 0.43190658 -0.2373541   0.24513614]
  [ 0.44747078 -0.14396888  0.16731513]
  [ 0.4785992  -0.07392997  0.11284041]]

 [[ 0.4708171  -0.01167315  0.29182875]
  [ 0.78210115  0.19844353  0.463035  ]
  [ 0.78988326  0.22957194  0.5953307 ]
  ...
  [ 0.22957194 -0.33073932  0.0272373 ]
  [ 0.22957194 -0.33852142  0.0350194 ]
  [ 0.27626455 -0.2373541   0.0661478 ]]

 [[ 0.5875486   0.12062252  0.540856  ]
  [ 0.78988326  0.24513614  0.463035  ]
  [ 0.69649804  0.19844353  0.43968868]
  ...
  [ 0.23735404 -0.2762646  -0.00389105]
  [ 0.22957194 -0.33073932  0.0583657 ]
  [ 0.4785992  -0.2607004   0.12062252]]

 ...

 [[ 0.11284041 -0.35408562  0.09727621]
  [ 0.5175097  -0.13618678  0.17509723]
  [ 0.83657587  0.22957194  0.4863813 ]
  ...
  [ 0.01945519 -0.34630352  0.18287933]
  [ 0.07392991 -0.37743193 -0.01945525]
  [ 0.35408556 -0.35408562 -0.066

tf.Tensor(
[[[ 0.33852136 -0.17509729  0.21400774]
  [ 0.12840462 -0.36186773  0.18287933]
  [ 0.37743187 -0.36186773 -0.05058366]
  ...
  [ 0.5564202  -0.19844359  0.14396882]
  [ 0.5719844  -0.17509729  0.08949411]
  [ 0.5252918  -0.17509729  0.20622563]]

 [[ 0.26848245 -0.44747084 -0.13618678]
  [ 0.21400774 -0.43968874 -0.14396888]
  [ 0.40077817 -0.36186773 -0.15175098]
  ...
  [ 0.22178984 -0.2762646   0.08949411]
  [ 0.33852136 -0.229572    0.12840462]
  [ 0.5797665  -0.08949417  0.22957194]]

 [[ 0.29182875 -0.35408562  0.0583657 ]
  [ 0.38521397 -0.31517512  0.0505836 ]
  [ 0.21400774 -0.40856034 -0.08171207]
  ...
  [ 0.20622563 -0.36186773  0.10505831]
  [ 0.33852136 -0.2062257   0.24513614]
  [ 0.6575875   0.12062252  0.43190658]]

 ...

 [[ 0.79766536  0.22957194  0.5097276 ]
  [ 0.29182875 -0.05836576  0.37743187]
  [-0.01945525 -0.37743193  0.11284041]
  ...
  [ 0.36964977 -0.2840467   0.08949411]
  [ 0.29182875 -0.2529183   0.10505831]
  [ 0.23735404 -0.31517512  0.159

tf.Tensor(
[[[ 0.5097276   0.0583657   0.30739295]
  [ 0.75875485  0.32295716  0.5875486 ]
  [ 0.5564202   0.12062252  0.41634238]
  ...
  [-0.36964983 -0.6264591  -0.2762646 ]
  [-0.5175097  -0.77431905 -0.47081715]
  [-0.6108949  -0.85214007 -0.5953307 ]]

 [[ 0.6420233   0.27626455  0.43968868]
  [ 0.5953307   0.24513614  0.40856028]
  [ 0.6031128   0.26070035  0.4708171 ]
  ...
  [-0.36186773 -0.6498054  -0.2373541 ]
  [-0.34630352 -0.71984434 -0.307393  ]
  [-0.34630352 -0.76653695 -0.35408562]]

 [[ 0.78988326  0.5875486   0.6264591 ]
  [ 0.78988326  0.540856    0.6342412 ]
  [ 0.71206224  0.40077817  0.5719844 ]
  ...
  [-0.49416345 -0.74319065 -0.33073932]
  [-0.33073932 -0.71206224 -0.2529183 ]
  [-0.12840468 -0.6108949  -0.13618678]]

 ...

 [[ 0.6031128   0.13618672  0.16731513]
  [ 0.66536963  0.08949411  0.29961085]
  [ 0.43968868 -0.12840468  0.12840462]
  ...
  [ 0.32295716 -0.03501946  0.32295716]
  [ 0.36964977 -0.05058366  0.18287933]
  [ 0.5252918   0.0583657   0.346

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
val_256 = tf.data.TFRecordDataset('dataset/ODG-tfrec-100-r08.tfrecords', compression_type='', buffer_size=buffer_mb<<20)
val_256 = val_256.map(parse_tfrecord_tf)#, num_parallel_calls=num_threads)
val_256 = val_256.take(64)#.batch(64)
val_imgs = list(val_256.as_numpy_iterator())

In [11]:
val_imgs

[array([[[ 0.6342412 , -0.33073932, -0.17509729],
         [ 0.6342412 , -0.2529183 , -0.11284047],
         [ 0.07392991, -0.5330739 , -0.05058366],
         ...,
         [ 0.74319065, -0.04280156,  0.24513614],
         [ 0.44747078, -0.37743193,  0.07392991],
         [ 0.6031128 , -0.41634244, -0.2140078 ]],
 
        [[ 0.40077817, -0.49416345, -0.12062258],
         [ 0.4941634 , -0.42412454, -0.2373541 ],
         [ 0.5330739 , -0.32295722, -0.10505837],
         ...,
         [ 0.4708171 , -0.48638135, -0.2217899 ],
         [ 0.4708171 , -0.47081715, -0.18287939],
         [ 0.4941634 , -0.38521403, -0.16731519]],
 
        [[ 0.27626455, -0.40856034,  0.00389099],
         [ 0.27626455, -0.38521403, -0.06614786],
         [ 0.29182875, -0.5175097 , -0.19844359],
         ...,
         [ 0.32295716, -0.48638135, -0.14396888],
         [ 0.08949411, -0.6031128 , -0.18287939],
         [ 0.6031128 , -0.36186773, -0.04280156]],
 
        ...,
 
        [[ 0.18287933, -0.45525295

In [104]:
buffer_mb       = 256
train_dataset = tf.data.TFRecordDataset('Molpax/ODG-tfrec-100-r03.tfrecords', compression_type='')
train_dataset = train_dataset.map(parse_tfrecord_tf)
i = 0
for train_data in train_dataset:
    i += 1
print(i)

11400


In [100]:
buffer_mb       = 256
train_dataset = tf.data.TFRecordDataset('a.tfrecords', compression_type='', buffer_size=buffer_mb<<20)
BATCH_SIZE = [128, 64, 64, 32, 32, 16, 8]
train_dataset = train_dataset.map(parse_tfrecord_tf)
for train_data in train_dataset:
    print(train_data)

tf.Tensor(
[[[-0.00389105  0.20622563 -0.03501946]
  [ 0.01167309  0.8832685   0.24513614]
  [ 0.24513614  0.9455253   0.33852136]
  [ 0.8988327   0.12840462  0.16731513]]

 [[-0.43190664  0.9455253  -0.618677  ]
  [-0.83657587  0.0505836  -0.2684825 ]
  [ 0.5642023   0.44747078  0.75875485]
  [ 0.5175097   0.44747078  0.39299607]]

 [[-0.07392997  0.32295716  0.6108949 ]
  [ 0.72762644  0.0272373   0.8754864 ]
  [ 0.6498054   0.74319065 -0.33852142]
  [ 0.26848245  0.74319065 -0.11284047]]

 [[-0.6575875   0.44747078  0.84435797]
  [-0.2762646   0.32295716 -0.2996109 ]
  [-0.36186773 -0.81322956 -0.2140078 ]
  [ 0.31517506  0.71984434 -0.5564202 ]]], shape=(4, 4, 3), dtype=float32)
